# Productionising the ML model
This notebook will walk through the process of creating, building and commiting the artifacts required to run the model developed in the Experimentation Notebook in production. 

## Environment Setup
**NOTE:** Set Project ID to your project  

In [1]:
PROJECT_ID = 'marina-test-proj'
PREFIX = PROJECT_ID
REGION = 'us-central1'

DATA_ROOT = 'gs://workshop-datasets/covertype/data_validation'
TRAINING_FILE_PATH = DATA_ROOT + '/training/dataset.csv'
VALIDATION_FILE_PATH = DATA_ROOT + '/evaluation/dataset.csv'

# Job dir for AI Platform Training
JOB_DIR_ROOT='gs://{}-artifact-store/jobs'.format(PREFIX)


NAMESPACE='kubeflow'
ZONE='us-central1-a'
ARTIFACT_STORE_URI='gs://{}-artifact-store'.format(PREFIX)
GCS_STAGING_PATH='{}/staging'.format(ARTIFACT_STORE_URI)
GKE_CLUSTER_NAME='{}-cluster'.format(PREFIX)

!gcloud container clusters get-credentials $GKE_CLUSTER_NAME --zone $ZONE
HOST_TEMP=!(kubectl describe configmap inverse-proxy-config -n $NAMESPACE | grep "googleusercontent.com")
INVERSE_PROXY_HOSTNAME=HOST_TEMP[0]



Fetching cluster endpoint and auth data.
kubeconfig entry generated for marina-test-proj-cluster.


## Imports

In [2]:
import json
import os
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

from googleapiclient import discovery
from googleapiclient import errors

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple

from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


## Import data set to BQ
Import the data set from cloud storage to BigQuery. A dataset is created and the table is imported under `covertype_data.covertype`

In [3]:
DATASET_LOCATION='US'
DATASET_ID='covertype_dataset'
TABLE_ID='covertype'
DATA_SOURCE='gs://workshop-datasets/covertype/full/dataset.csv'
SCHEMA='Elevation:INTEGER,\
Aspect:INTEGER,\
Slope:INTEGER,\
Horizontal_Distance_To_Hydrology:INTEGER,\
Vertical_Distance_To_Hydrology:INTEGER,\
Horizontal_Distance_To_Roadways:INTEGER,\
Hillshade_9am:INTEGER,\
Hillshade_Noon:INTEGER,\
Hillshade_3pm:INTEGER,\
Horizontal_Distance_To_Fire_Points:INTEGER,\
Wilderness_Area:STRING,\
Soil_Type:STRING,\
Cover_Type:INTEGER'

!bq --location=$DATASET_LOCATION --project_id=$PROJECT_ID mk --dataset $DATASET_ID
!bq --project_id=$PROJECT_ID --dataset_id=$DATASET_ID load \
--source_format=CSV \
--skip_leading_rows=1 \
--replace \
$TABLE_ID \
$DATA_SOURCE \
$SCHEMA

BigQuery error in mk operation: Dataset 'marina-test-proj:covertype_dataset'
already exists.
Waiting on bqjob_r1b007fafba86bce9_000001788184eeaa_1 ... (5s) Current status: DONE   


## Prepare the training application.
Now that the data is hosted in BQ and we have created the KFP CLI Builder image, the next step is to create the training application. Start by creating the master pipeline folder and nested folders to host the model script, trainer image docker and the base image docker. 

In [7]:
!pwd
#os.chdir('../02_demo_CICD')

/home/mlops-demo/02_demo_CICD


In [8]:
PIPELINE_APP_FOLDER ='pipeline'
TRAINING_APP_FOLDER = 'pipeline/trainer_image'
BASE_IMAGE_FOLDER='pipeline/base_image'
os.makedirs(PIPELINE_APP_FOLDER, exist_ok=True)
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)
os.makedirs(BASE_IMAGE_FOLDER, exist_ok=True)

### Write the training script. 

The script written in the Experimentation Notebook, which process the data and trains the classification model, is written as a training script `train.py` in the training image folder. In addition to the model written during experimentation, an additional `hypertune` function is created which allows for a training job to be run with multiple parameters. This will  run multiple models with a range of parameters on CAIP training platform. 

In [9]:
%%writefile {TRAINING_APP_FOLDER}/train.py
"""Covertype Classifier trainer script."""

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, hptune):
    
  df_train = pd.read_csv(training_dataset_path)
  df_validation = pd.read_csv(validation_dataset_path)
    
  if not hptune:
    df_train = pd.concat([df_train, df_validation])

  numeric_feature_indexes = slice(0, 10)
  categorical_feature_indexes = slice(10, 12)

  preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

  pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log'))
  ])
    
  num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}
  df_train = df_train.astype(num_features_type_map)
  df_validation = df_validation.astype(num_features_type_map) 

  print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
  X_train = df_train.drop('Cover_Type', axis=1)
  y_train = df_train['Cover_Type']
  
  pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
  pipeline.fit(X_train, y_train)
  
  if hptune:
    X_validation = df_validation.drop('Cover_Type', axis=1)
    y_validation = df_validation['Cover_Type']
    accuracy = pipeline.score(X_validation, y_validation)
    print('Model accuracy: {}'.format(accuracy))
    # Log it with hypertune
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=accuracy
    )

  # Save the model
  if not hptune:
    model_filename = 'model.pkl'
    with open(model_filename, 'wb') as model_file:
        pickle.dump(pipeline, model_file)
    gcs_model_path = "{}/{}".format(job_dir, model_filename)
    subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
    print("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  fire.Fire(train_evaluate)

Writing pipeline/trainer_image/train.py


# Write pipeline script and accompanying helper components 
Now that we have images with the base dependancies and the training script built and saved into the container registry, the next step is to write the pipeline script `covertype_training_pipeline.py` which defines the pipeline which will be deployed to KFP. This file sets hypertuning settings and uses both pre-built components and custome built components defined in a seperate `helper_components.py`.  


- Pre-build components. The pipeline uses the following pre-build components that are included with KFP distribution:
    - BigQuery query component
    - AI Platform Training component
    - AI Platform Deploy component
- Custom components. The pipeline uses two custom helper components that encapsulate functionality not available in any of the pre-build components. The components are implemented using the KFP SDK's Lightweight Python Components mechanism. The code for the components is in the helper_components.py file:
    - Retrieve Best Run- This component retrieves the tuning metric and hyperparameter values for the best run of the AI Platform Training hyperparameter tuning job.
    - Evaluate Model - This component evaluates the sklearn trained model using a provided metric and a testing dataset.

The workflow implemented by the pipeline is defined using a Python based KFP Domain Specific Language (DSL). The pipeline's DSL is in the covertype_training_pipeline.py file.

In [10]:
%%writefile {PIPELINE_APP_FOLDER}/covertype_training_pipeline.py

"""KFP pipeline orchestrating BigQuery and Cloud AI Platform services."""

import os

from helper_components import evaluate_model
from helper_components import retrieve_best_run
from jinja2 import Template
import kfp
from kfp.components import func_to_container_op
from kfp.dsl.types import Dict
from kfp.dsl.types import GCPProjectID
from kfp.dsl.types import GCPRegion
from kfp.dsl.types import GCSPath
from kfp.dsl.types import String
from kfp.gcp import use_gcp_secret

# Defaults and environment settings
BASE_IMAGE = os.getenv('BASE_IMAGE')
TRAINER_IMAGE = os.getenv('TRAINER_IMAGE')
RUNTIME_VERSION = os.getenv('RUNTIME_VERSION')
PYTHON_VERSION = os.getenv('PYTHON_VERSION')
COMPONENT_URL_SEARCH_PREFIX = os.getenv('COMPONENT_URL_SEARCH_PREFIX')


TRAINING_FILE_PATH = 'datasets/training/data.csv'
VALIDATION_FILE_PATH = 'datasets/validation/data.csv'
TESTING_FILE_PATH = 'datasets/testing/data.csv'

# Parameter defaults
SPLITS_DATASET_ID = 'splits'
HYPERTUNE_SETTINGS = """
{
    "hyperparameters":  {
        "goal": "MAXIMIZE",
        "maxTrials": 6,
        "maxParallelTrials": 3,
        "hyperparameterMetricTag": "accuracy",
        "enableTrialEarlyStopping": True,
        "params": [
            {
                "parameterName": "max_iter",
                "type": "DISCRETE",
                "discreteValues": [500, 1000]
            },
            {
                "parameterName": "alpha",
                "type": "DOUBLE",
                "minValue": 0.0001,
                "maxValue": 0.001,
                "scaleType": "UNIT_LINEAR_SCALE"
            }
        ]
    }
}
"""


# Helper functions
def generate_sampling_query(source_table_name, num_lots, lots):
  """Prepares the data sampling query."""

  sampling_query_template = """
       SELECT *
       FROM 
           `{{ source_table }}` AS cover
       WHERE 
       MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) IN ({{ lots }})
       """
  query = Template(sampling_query_template).render(
      source_table=source_table_name, num_lots=num_lots, lots=str(lots)[1:-1])

  return query


# Create component factories
component_store = kfp.components.ComponentStore(
    local_search_paths=None, url_search_prefixes=[COMPONENT_URL_SEARCH_PREFIX])

bigquery_query_op = component_store.load_component('bigquery/query')
mlengine_train_op = component_store.load_component('ml_engine/train')
mlengine_deploy_op = component_store.load_component('ml_engine/deploy')
retrieve_best_run_op = func_to_container_op(
    retrieve_best_run, base_image=BASE_IMAGE)
evaluate_model_op = func_to_container_op(evaluate_model, base_image=BASE_IMAGE)


@kfp.dsl.pipeline(
    name='Covertype Classifier Training',
    description='The pipeline training and deploying the Covertype classifierpipeline_yaml'
)
def covertype_train(project_id: GCPProjectID,
                    region: GCPRegion,
                    source_table_name: String,
                    gcs_root: GCSPath,
                    dataset_id: str,
                    evaluation_metric_name: str,
                    evaluation_metric_threshold: float,
                    model_id: str,
                    version_id: str,
                    replace_existing_version: bool,
                    hypertune_settings: Dict = HYPERTUNE_SETTINGS,
                    dataset_location: str = 'US'):
  """Orchestrates training and deployment of an sklearn model."""

  # Create the training split
  query = generate_sampling_query(
      source_table_name=source_table_name, num_lots=10, lots=[1, 2, 3, 4])

  training_file_path = '{}/{}'.format(gcs_root, TRAINING_FILE_PATH)

  create_training_split = bigquery_query_op(
      query=query,
      project_id=project_id,
      dataset_id=dataset_id,
      table_id='',
      output_gcs_path=training_file_path,
      dataset_location=dataset_location)

  # Create the validation split
  query = generate_sampling_query(
      source_table_name=source_table_name, num_lots=10, lots=[8])

  validation_file_path = '{}/{}'.format(gcs_root, VALIDATION_FILE_PATH)

  create_validation_split = bigquery_query_op(
      query=query,
      project_id=project_id,
      dataset_id=dataset_id,
      table_id='',
      output_gcs_path=validation_file_path,
      dataset_location=dataset_location)

  # Create the testing split
  query = generate_sampling_query(
      source_table_name=source_table_name, num_lots=10, lots=[9])

  testing_file_path = '{}/{}'.format(gcs_root, TESTING_FILE_PATH)

  create_testing_split = bigquery_query_op(
      query=query,
      project_id=project_id,
      dataset_id=dataset_id,
      table_id='',
      output_gcs_path=testing_file_path,
      dataset_location=dataset_location)

  # Tune hyperparameters
  tune_args = [
      '--training_dataset_path',
      create_training_split.outputs['output_gcs_path'],
      '--validation_dataset_path',
      create_validation_split.outputs['output_gcs_path'], '--hptune', 'True'
  ]

  job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir/hypertune',
                              kfp.dsl.RUN_ID_PLACEHOLDER)

  hypertune = mlengine_train_op(
      project_id=project_id,
      region=region,
      master_image_uri=TRAINER_IMAGE,
      job_dir=job_dir,
      args=tune_args,
      training_input=hypertune_settings)

  # Retrieve the best trial
  get_best_trial = retrieve_best_run_op(project_id, hypertune.outputs['job_id'])

  # Train the model on a combined training and validation datasets
  job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir', kfp.dsl.RUN_ID_PLACEHOLDER)

  train_args = [
      '--training_dataset_path',
      create_training_split.outputs['output_gcs_path'],
      '--validation_dataset_path',
      create_validation_split.outputs['output_gcs_path'], '--alpha',
      get_best_trial.outputs['alpha'], '--max_iter',
      get_best_trial.outputs['max_iter'], '--hptune', 'False'
  ]

  train_model = mlengine_train_op(
      project_id=project_id,
      region=region,
      master_image_uri=TRAINER_IMAGE,
      job_dir=job_dir,
      args=train_args)

  # Evaluate the model on the testing split
  eval_model = evaluate_model_op(
      dataset_path=str(create_testing_split.outputs['output_gcs_path']),
      model_path=str(train_model.outputs['job_dir']),
      metric_name=evaluation_metric_name)

  # Deploy the model if the primary metric is better than threshold
  with kfp.dsl.Condition(
      eval_model.outputs['metric_value'] > evaluation_metric_threshold):
    deploy_model = mlengine_deploy_op(
        model_uri=train_model.outputs['job_dir'],
        project_id=project_id,
        model_id=model_id,
        version_id=version_id,
        runtime_version=RUNTIME_VERSION,
        python_version=PYTHON_VERSION,
        replace_existing_version=replace_existing_version)

  kfp.dsl.get_pipeline_conf().add_op_transformer(use_gcp_secret('user-gcp-sa'))


Writing pipeline/covertype_training_pipeline.py


## Write the custom 'helper' components 

In [11]:
%%writefile {PIPELINE_APP_FOLDER}/helper_components.py

"""Helper components."""

from typing import NamedTuple


def retrieve_best_run(
    project_id: str, job_id: str
) -> NamedTuple('Outputs', [('metric_value', float), ('alpha', float),
                            ('max_iter', int)]):
  """Retrieves the parameters of the best Hypertune run."""

  from googleapiclient import discovery
  from googleapiclient import errors

  ml = discovery.build('ml', 'v1')

  job_name = 'projects/{}/jobs/{}'.format(project_id, job_id)
  request = ml.projects().jobs().get(name=job_name)

  try:
    response = request.execute()
  except errors.HttpError as err:
    print(err)
  except:
    print('Unexpected error')

  print(response)

  best_trial = response['trainingOutput']['trials'][0]

  metric_value = best_trial['finalMetric']['objectiveValue']
  alpha = float(best_trial['hyperparameters']['alpha'])
  max_iter = int(best_trial['hyperparameters']['max_iter'])

  return (metric_value, alpha, max_iter)


def evaluate_model(
    dataset_path: str, model_path: str, metric_name: str
) -> NamedTuple('Outputs', [('metric_name', str), ('metric_value', float),
                            ('mlpipeline_metrics', 'Metrics')]):
  """Evaluates a trained sklearn model."""
  #import joblib
  import pickle
  import json
  import pandas as pd
  import subprocess
  import sys

  from sklearn.metrics import accuracy_score, recall_score

  df_test = pd.read_csv(dataset_path)

  X_test = df_test.drop('Cover_Type', axis=1)
  y_test = df_test['Cover_Type']

  # Copy the model from GCS
  model_filename = 'model.pkl'
  gcs_model_filepath = '{}/{}'.format(model_path, model_filename)
  print(gcs_model_filepath)
  subprocess.check_call(['gsutil', 'cp', gcs_model_filepath, model_filename],
                        stderr=sys.stdout)

  with open(model_filename, 'rb') as model_file:
    model = pickle.load(model_file)

  y_hat = model.predict(X_test)

  if metric_name == 'accuracy':
    metric_value = accuracy_score(y_test, y_hat)
  elif metric_name == 'recall':
    metric_value = recall_score(y_test, y_hat)
  else:
    metric_name = 'N/A'
    metric_value = 0

  # Export the metric
  metrics = {
      'metrics': [{
          'name': metric_name,
          'numberValue': float(metric_value)
      }]
  }

  return (metric_name, metric_value, json.dumps(metrics))


Writing pipeline/helper_components.py


# Creating KFP CLI builder

In [12]:
%%writefile Dockerfile 
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install https://storage.googleapis.com/ml-pipeline/release/0.1.36/kfp.tar.gz 

ENTRYPOINT ["/bin/bash"]

Writing Dockerfile


In [13]:
IMAGE_NAME='kfp-cli'
IMAGE_URI="gcr.io/{}/{}:latest".format(PROJECT_ID,IMAGE_NAME)
!gcloud builds submit --timeout 15m --tag {IMAGE_URI} 

Creating temporary tarball archive of 6 file(s) totalling 87.3 KiB before compression.
Uploading tarball of [.] to [gs://marina-test-proj_cloudbuild/source/1617080849.6-e4815079450742d781e0fbe7960a32a6.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/marina-test-proj/builds/bfba6d74-7ce6-47fc-b987-c486b91a0b17].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/bfba6d74-7ce6-47fc-b987-c486b91a0b17?project=989115307566].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "bfba6d74-7ce6-47fc-b987-c486b91a0b17"

FETCHSOURCE
Fetching storage object: gs://marina-test-proj_cloudbuild/source/1617080849.6-e4815079450742d781e0fbe7960a32a6.tgz#1617080849827269
Copying gs://marina-test-proj_cloudbuild/source/1617080849.6-e4815079450742d781e0fbe7960a32a6.tgz#1617080849827269...
/ [1 files][ 19.4 KiB/ 19.4 KiB]                                                
Operation completed over 1 objects/19.4 KiB.                 

### Package the script into a docker image.

The docker images used for the training are based off the image `mlops-dev:latest` created during the inital set up of the environment. Since the AI Platform Notebook instance is based on the `mlops-dev:latest` image we use the same image as a base for the training image. 


We first write a base image dockerfile which replicates the image used for the Notebook. Then we write a training dockerfile which uses the same base image and add the `train.py` to the image. 


**NOTE:** Make sure to update the URI for the image so that it points to your project's **Container Registry**. i.e. `FROM gcr.io/PROJECT_ID/kfp-cli:latest` 

In [14]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/marinas-demo/mlops-dev:latest
RUN pip install -U fire cloudml-hypertune
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing pipeline/trainer_image/Dockerfile


In [15]:
%%writefile {BASE_IMAGE_FOLDER}/Dockerfile
FROM gcr.io/marinas-demo/mlops-dev:latest

Writing pipeline/base_image/Dockerfile


## Create cloud build config file
This file is used by **Cloud Build** to create all container images required and deploys the pipeline to kubeflow pipelines

In [16]:
%%writefile cloudbuild.yaml

steps:
# Build the trainer image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME', '.']
  dir: $_PIPELINE_FOLDER/trainer_image
  
# Build the base image for lightweight components
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME', '.']
  dir: $_PIPELINE_FOLDER/base_image

# Compile the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli'
  args:
  - '-c'
  - |
    dsl-compile --py $_PIPELINE_DSL --output $_PIPELINE_PACKAGE
  env:
  - 'BASE_IMAGE=gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME'
  - 'TRAINER_IMAGE=gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME'
  - 'RUNTIME_VERSION=$_RUNTIME_VERSION'
  - 'PYTHON_VERSION=$_PYTHON_VERSION'
  - 'COMPONENT_URL_SEARCH_PREFIX=$_COMPONENT_URL_SEARCH_PREFIX'
  dir: $_PIPELINE_FOLDER
  
 # Upload the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli'
  args:
  - '-c'
  - |
    kfp --endpoint $_INVERTING_PROXY_HOST pipeline upload -p ${_PIPELINE_NAME}_$TAG_NAME $_PIPELINE_PACKAGE
  dir: $_PIPELINE_FOLDER


# Push the images to Container Registry 
images: ['gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME', 'gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME']

Writing cloudbuild.yaml


# Commit to Cloud Source Repo

### Set up Source Repo

In [23]:
REPO_NAME='{}-test'.format(PROJECT_ID)

Open terminal in Jupyter notebook and set up an authentication channel to cloud source repo. Run the gcloud init command and follow prompts, create a new source repo with the name of your project ID  
`gcloud init && git config credential.helper gcloud.sh` - select option 2, then yes, follow the link, copy the verification code back in the terminal command. Once verified, select [1] Enter project ID and type your project ID, then select no for configure region.

`gcloud source repos create REPO_NAME` - you will need to specify your REPO_NAME 

*This step is simpler if done in terminal*


In [24]:
!cd /home/mlops-demo
!git remote add {REPO_NAME} https://source.developers.google.com/p/{PROJECT_ID}/r/{REPO_NAME}

fatal: remote marina-test-proj-test already exists.


In [26]:
!git add .

In [21]:
!git commit -m "Upload ML" 

On branch master
Your branch is ahead of 'origin/master' by 2 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
	modified:   ../01_Experimentation.ipynb

no changes added to commit


In [22]:
!git push --all {REPO_NAME}

fatal: remote error: Git repository not found


# Create Cloud Build trigger
This will cause the images to build and will deploy the pipeline apon a push to the source repository

## Write a trigger build configuration file
This file is used by cloud trigger, to pass the variables required by cloud build. 

**NOTE** Ensure to change the `Project_ID`, `repoName` and `_INVERTING_PROXY_HOST` variables to reflect your environment

In [ ]:
print(INVERSE_PROXY_HOSTNAME)

In [ ]:
%%writefile trigger_build.yaml

description: triggers cloud build upon push
name: trigger-build
filename: 02_demo_CICD/cloudbuild.yaml
triggerTemplate:
    projectId: marinas-demo
    repoName: marinas-demo-test
    branchName: master
substitutions:
    _INVERTING_PROXY_HOST: 4bcc62bc46b37077-dot-us-central1.notebooks.googleusercontent.com
    _TRAINER_IMAGE_NAME: trainer_image
    _BASE_IMAGE_NAME: base_image
    TAG_NAME: test
    _PIPELINE_FOLDER: 02_demo_CICD/pipeline
    _PIPELINE_DSL: covertype_training_pipeline.py
    _PIPELINE_PACKAGE: covertype_training_pipeline.yaml
    _PIPELINE_NAME: covertype_training_deployment
    _RUNTIME_VERSION: "1.14"
    _PYTHON_VERSION: "3.5"
    _COMPONENT_URL_SEARCH_PREFIX: https://raw.githubusercontent.com/kubeflow/pipelines/0.1.36/components/gcp/


This command creates a **Cloud Build** trigger upon any new code committed to the master of our repository. 

**NOTE**: Only run this command once or it will create multiple triggers

In [ ]:
!gcloud beta builds triggers create cloud-source-repositories --trigger-config=trigger_build.yaml --project {PROJECT_ID}

## Run trigger on push
We will now push code to our repository, which will automatically trigger a cloud build of the new container images

In [ ]:
!git add .
!git commit -m "Upload ML"
!git push --all {REPO_NAME}   

In [ ]:
!gcloud builds list --ongoing

In [ ]:
!kfp --endpoint {INVERSE_PROXY_HOSTNAME} pipeline list

# Viewing the pipeline
The deployed pipeline can be viewed through the Kubeflow Pipeline UI given at the URL below. 

In [ ]:
print('https://{}'.format(INVERSE_PROXY_HOSTNAME))

## Run Experiment 
Now that the pipeline is deployed we want to run an experiment, this will cause the pipeline to run, pulling the data from bigquery and splitting it, training the models, evaluating them and deploy the best performing model. This experiment takes approximately an hour to execute and will result in a deployed model which can be interacted with through GCP's AI platform predicting service. 

**NOTE:** Change the PIPELINE_ID to reflect the ID copied from above.  

In [ ]:
PIPELINE_ID='ab90cc6c-3f00-480c-bea3-bd1959e7394b'

EXPERIMENT_NAME='Covertype_Classifier_Training'
RUN_ID='Run_001'
SOURCE_TABLE='covertype_dataset.covertype'
DATASET_ID='splits'
EVALUATION_METRIC='accuracy'
EVALUATION_METRIC_THRESHOLD='0.69'
MODEL_ID='covertype_classifier'
VERSION_ID='v01'
REPLACE_EXISTING_VERSION=True

In [ ]:
!kfp --endpoint {INVERSE_PROXY_HOSTNAME} run submit \
-e Covertype_Classifier_Training \
-r {RUN_ID} \
-p {PIPELINE_ID} \
project_id={PROJECT_ID} \
gcs_root={GCS_STAGING_PATH} \
region={REGION} \
source_table_name={SOURCE_TABLE} \
dataset_id={DATASET_ID} \
evaluation_metric_name={EVALUATION_METRIC} \
evaluation_metric_threshold={EVALUATION_METRIC_THRESHOLD} \
model_id={MODEL_ID} \
version_id={VERSION_ID} \
replace_existing_version={REPLACE_EXISTING_VERSION}

## Testing model
To test the model we can use the AI platforms prediction API to ask for a prediction based on a JSON input aternatively we can use the prediction UI and input: *{"instances":[[2395,0,0,60,6,1170,218,238,156,1054,"Cache","C2717"]]}* in the test case window.

We write a prediction JSON file with a set of data points, the correct cover types are 3 and 2 respectively.

In [ ]:
%%writefile predict.json
[2395,0,0,60,6,1170,218,238,156,1054,"Cache","C2717"]
[2756,135,0,85,14,1608,219,238,156,2451,"Rawah","C4744"]


In [ ]:
INPUT_DATA_FILE="./predict.json"

!gcloud ai-platform predict --model {MODEL_ID} \
  --version {VERSION_ID} \
  --json-instances {INPUT_DATA_FILE}

## Clean Up Demo

In [ ]:
#!rm -r pipeline/ cloudbuild.yaml build_pipeline.sh Dockerfile